In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
from model import FlowNet
from dataset import Scene_Flow_disparity

In [ ]:
dataset = Scene_Flow_disparity()

In [ ]:
print(len(dataset.data_paths))
print()
iteration = 0
for dir_ in dataset.data_paths:
    iteration += 1
    if iteration == 200:
        break
    if dataset.data(dir_) is not None:
        print(dir_)

In [ ]:
import time
import datetime

flownet = FlowNet(
    img_height = 540, 
    img_width = 960, 
    img_depth = 3, 
    learning_rate = 0.1**4).inference('simple')

start=time.clock()

In [ ]:
train_x = []
train_y = []
for dir_ in dataset.data_paths:
    if dataset.data(dir_) is not None:
        left_image, right_image, ground_truth = dataset.data(dir_)

        train_x.append(np.append(left_image, right_image, axis = 2))
        train_y.append(ground_truth)
        
    if len(train_y_list) == 1000:                       
        hist = flownet.fit(np.array(train_x), np.array(train_y), batch_size=8, epochs=100, validation_split=0.2)
        flownet.save_weights('./checkpoints/flownetSimple_for_depth.hdf5')
        train_x.clear()
        train_y.clear()

In [ ]:
runtime_sec = time.clock() - start
m, s = divmod(runtime_sec, 60)
h, m = divmod(m, 60)
d, h = divmod(h, 24)
print('runtime : %d days %d:%02d:%02d' % (d, h, m, s))

In [ ]:
flownet.save_weights('./checkpoints/flownetSimple_for_depth.hdf5')